In [1]:
import recordlinkage
import pandas

print("reading files")

filepath = './datasets/%s'
outfile = './dlbp_scholar_matches.csv'

# if True, uses a smaller version of the dataset for faster computation
debug = True

filename_postfix  = ''
if debug:
    filename_postfix  = '_mini'

dataDBLP = pandas.read_csv(filepath % ('DBLP-Scholar/DBLP1%s.csv' % filename_postfix))
dataScholar = pandas.read_csv(filepath % 'DBLP-Scholar/Scholar%s.csv' % filename_postfix)

print("files read")
print('len of DBLP: %d' % len(dataDBLP))
print('len of Scholar: %d' % len(dataScholar))

reading files
files read
len of DBLP: 999
len of Scholar: 999


In [2]:
indexer = recordlinkage.Index()
indexer.block('year')
candidate_links = indexer.index(dataDBLP, dataScholar)

# print(candidate_links)
print(len(candidate_links))

print('computing match...')

compare_cl = recordlinkage.Compare()
compare_cl.string('title', 'title', threshold=0.85, label='title_matched')
result = compare_cl.compute(candidate_links, dataDBLP, dataScholar)
result = result.reset_index()

print()
print('match result (1 means it\'s a match):')
result["title_matched"].reset_index().drop(columns=["index"]).sum(axis=1).value_counts().sort_index(ascending=False)
# print(features.sum(axis=0).value_counts())

23853
computing match...

match result (1 means it's a match):


1.0       11
0.0    23842
dtype: int64

In [3]:
# postprocessing - prepare for merging with original datasets
result = result.reset_index()
dataDBLP = dataDBLP.reset_index()
dataScholar = dataScholar.reset_index()

result["dblp_index"] = result["level_0"]
result["scholar_index"] = result["level_1"]
result

,index,level_0,level_1,title_matched,dblp_index,scholar_index
0,0,0,4,0.0,0,4
1,1,0,15,0.0,0,15
2,2,0,75,0.0,0,75
3,3,0,99,0.0,0,99
4,4,0,141,0.0,0,141
...,...,...,...,...,...,...
23848,23848,767,835,0.0,767,835
23849,23849,767,840,0.0,767,840
23850,23850,767,908,0.0,767,908
23851,23851,767,912,0.0,767,912


In [4]:
pos_match = result[result["title_matched"] == 1]
pos_match

,index,level_0,level_1,title_matched,dblp_index,scholar_index
1649,1649,852,406,1.0,852,406
2365,2365,35,729,1.0,35,729
3150,3150,229,811,1.0,229,811
3265,3265,245,25,1.0,245,25
3299,3299,248,191,1.0,248,191
6897,6897,141,370,1.0,141,370
13000,13000,412,843,1.0,412,843
13951,13951,417,435,1.0,417,435
14058,14058,428,166,1.0,428,166
15713,15713,958,295,1.0,958,295


In [5]:
matched_data = pos_match.merge(dataDBLP, left_on="dblp_index", right_on="index")
matched_data = matched_data.merge(dataScholar, left_on="scholar_index", right_on="index")

# cleanup
matched_data["id_DLBP"] = matched_data["id_x"]
matched_data["id_Scholar"] = matched_data["id_y"]

matched_data_trimmed = matched_data[["id_DLBP", "id_Scholar", "title_x", "title_y"]]

matched_data_trimmed.to_csv(outfile)
matched_data_trimmed

,id_DLBP,id_Scholar,title_x,title_y
0,journals/sigmod/DrewHKKLLLSW95,9fcJjh5vHn8J,Data and Knowledge Base Research at Hong Kong ...,Data and Knowledge Base Research at Hong Kong ...
1,journals/sigmod/Camps02,JclQFC-1s18J,From Ternary Relationship to Relational Tables...,From Ternary Relationship to Relational Tables...
2,journals/sigmod/SommerGB02,8l5_eKZcCh8J,The n-Tier Hub Technology,The n-Tier Hub Technology
3,journals/sigmod/Stonebraker02,0HMk-YUh4i8J,Too Much Middleware,Too Much Middleware
4,journals/sigmod/GobleR02,url:http://portal.acm.org/ft_gateway.cfm%3Fid%...,Report on the 18th British National Conference...,Report on the 18th British National Conference...
5,journals/sigmod/ChienTZ01,cjBpI3TcSWsJ,XML Document Versioning,XML Document Versioning
6,conf/sigmod/KornMZ03,6VAQeD6muakJ,IPSOFACTO: A Visual Correlation Tool for Aggre...,IPSOFACTO: A Visual Correlation Tool for Aggre...
7,journals/sigmod/ShethBS99,NLklGN91cZgJ,VideoAnywhere: A System for Searching and Mana...,VideoAnywhere: A System for Searching and Mana...
8,journals/sigmod/CareyS99,url:http://portal.acm.org/ft_gateway.cfm%3Fid%...,NSF Workshop on Industrial/Academic Cooperatio...,NSF Workshop on Industrial/Academic Cooperatio...
9,conf/sigmod/AdaliSS99,c2dH5RxNj6kJ,A Multimedia Presentation Algebra,A Multimedia Presentation Algebra
